In [1]:
import findspark
findspark.init('/home/osboxes/SparkClass/spark-2.4.3-bin-hadoop2.7')
import pyspark
import os

In [2]:
myPath = os.path.join('/home', 'osboxes', 'SparkClass', 'Spark_for_Machine_Learning', 'Tree_Methods')

print(myPath)

os.chdir(myPath)

/home/osboxes/SparkClass/Spark_for_Machine_Learning/Tree_Methods


In [3]:
os.listdir()

['Tree Methods Code Along.ipynb',
 'Tree_Methods_Doc_Example.ipynb',
 '.ipynb_checkpoints',
 'dog_food.csv',
 'College.csv',
 'sample_libsvm_data.txt',
 'Tree_Methods_Consulting_Project.ipynb',
 'Tree_Methods_Consulting_Project_SOLUTION.ipynb']

In [4]:
# set up the session info
# import tree-based classifiers

from pyspark.sql import SparkSession

name = 'tree'

spark = SparkSession.builder.appName(name).getOrCreate()

from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier, GBTClassifier, DecisionTreeClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [5]:
# Load training data
data = spark.read.csv('College.csv',inferSchema=True,header=True)
data.printSchema()

root
 |-- School: string (nullable = true)
 |-- Private: string (nullable = true)
 |-- Apps: integer (nullable = true)
 |-- Accept: integer (nullable = true)
 |-- Enroll: integer (nullable = true)
 |-- Top10perc: integer (nullable = true)
 |-- Top25perc: integer (nullable = true)
 |-- F_Undergrad: integer (nullable = true)
 |-- P_Undergrad: integer (nullable = true)
 |-- Outstate: integer (nullable = true)
 |-- Room_Board: integer (nullable = true)
 |-- Books: integer (nullable = true)
 |-- Personal: integer (nullable = true)
 |-- PhD: integer (nullable = true)
 |-- Terminal: integer (nullable = true)
 |-- S_F_Ratio: double (nullable = true)
 |-- perc_alumni: integer (nullable = true)
 |-- Expend: integer (nullable = true)
 |-- Grad_Rate: integer (nullable = true)



In [6]:
data.head(1)

[Row(School='Abilene Christian University', Private='Yes', Apps=1660, Accept=1232, Enroll=721, Top10perc=23, Top25perc=52, F_Undergrad=2885, P_Undergrad=537, Outstate=7440, Room_Board=3300, Books=450, Personal=2200, PhD=70, Terminal=78, S_F_Ratio=18.1, perc_alumni=12, Expend=7041, Grad_Rate=60)]

In [9]:
# Import VectorAssembler and Vectors

from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [10]:
data.columns

['School',
 'Private',
 'Apps',
 'Accept',
 'Enroll',
 'Top10perc',
 'Top25perc',
 'F_Undergrad',
 'P_Undergrad',
 'Outstate',
 'Room_Board',
 'Books',
 'Personal',
 'PhD',
 'Terminal',
 'S_F_Ratio',
 'perc_alumni',
 'Expend',
 'Grad_Rate']

In [11]:
# create the assembler from the columns 
assembler = VectorAssembler(inputCols=['Apps',
                                       'Accept',
                                       'Enroll',
                                       'Top10perc',
                                       'Top25perc',
                                       'F_Undergrad',
                                       'P_Undergrad',
                                       'Outstate',
                                       'Room_Board',
                                       'Books',
                                       'Personal',
                                       'PhD',
                                       'Terminal',
                                       'S_F_Ratio',
                                       'perc_alumni',
                                       'Expend',
                                       'Grad_Rate'],
                            outputCol="features")

In [12]:
# perform the transformation

output = assembler.transform(data)

In [13]:
# get string indexer for 'Private' being yes or no to 1,0

from pyspark.ml.feature import StringIndexer

In [15]:
# make indexer and transform

indexer = StringIndexer(inputCol="Private", outputCol="PrivateIndex")
output_fixed = indexer.fit(output).transform(output)

In [16]:
# shows all of the columns in the data

output_fixed.printSchema()

root
 |-- School: string (nullable = true)
 |-- Private: string (nullable = true)
 |-- Apps: integer (nullable = true)
 |-- Accept: integer (nullable = true)
 |-- Enroll: integer (nullable = true)
 |-- Top10perc: integer (nullable = true)
 |-- Top25perc: integer (nullable = true)
 |-- F_Undergrad: integer (nullable = true)
 |-- P_Undergrad: integer (nullable = true)
 |-- Outstate: integer (nullable = true)
 |-- Room_Board: integer (nullable = true)
 |-- Books: integer (nullable = true)
 |-- Personal: integer (nullable = true)
 |-- PhD: integer (nullable = true)
 |-- Terminal: integer (nullable = true)
 |-- S_F_Ratio: double (nullable = true)
 |-- perc_alumni: integer (nullable = true)
 |-- Expend: integer (nullable = true)
 |-- Grad_Rate: integer (nullable = true)
 |-- features: vector (nullable = true)
 |-- PrivateIndex: double (nullable = false)



In [17]:
final_data = output_fixed.select("features",'PrivateIndex')

In [18]:
train, test = final_data.randomSplit([0.7, 0.3])

In [19]:
# alternative for regression

from pyspark.ml.regression import RandomForestRegressor

In [22]:
# work with the pipeline

from pyspark.ml import Pipeline

In [21]:
dtc = DecisionTreeClassifier(labelCol='PrivateIndex', featuresCol='features')
rfc = RandomForestClassifier(labelCol='PrivateIndex', featuresCol='features')
gbt = GBTClassifier(labelCol='PrivateIndex',featuresCol='features')

In [24]:
# Train the models

dtc_model = dtc.fit(train)
rfc_model = rfc.fit(train)
gbt_model = gbt.fit(train)

In [26]:
# make the predictions

dtc_predictions = dtc_model.transform(test)
rfc_predictions = rfc_model.transform(test)
gbt_predictions = gbt_model.transform(test)

In [27]:
# make the accuracy metric
# Select (prediction, true label) and compute test error

acc_evaluator = MulticlassClassificationEvaluator(labelCol="PrivateIndex",
                                                  predictionCol="prediction",
                                                  metricName="accuracy")

In [28]:
# apply the accuracy function
dtc_acc = acc_evaluator.evaluate(dtc_predictions)
rfc_acc = acc_evaluator.evaluate(rfc_predictions)
gbt_acc = acc_evaluator.evaluate(gbt_predictions)

In [29]:
# set up output

print("Here are the results!")
print('-'*80)
print('A single decision tree had an accuracy of: {0:2.2f}%'.format(dtc_acc*100))
print('-'*80)
print('A random forest ensemble had an accuracy of: {0:2.2f}%'.format(rfc_acc*100))
print('-'*80)
print('A ensemble using GBT had an accuracy of: {0:2.2f}%'.format(gbt_acc*100))

Here are the results!
--------------------------------------------------------------------------------
A single decision tree had an accuracy of: 92.73%
--------------------------------------------------------------------------------
A random forest ensemble had an accuracy of: 94.09%
--------------------------------------------------------------------------------
A ensemble using GBT had an accuracy of: 92.27%


In [33]:
# make a binary calssification evaluator AUC
from pyspark.ml.evaluation import BinaryClassificationEvaluator

auc = BinaryClassificationEvaluator(labelCol = 'PrivateIndex',
                                    rawPredictionCol="prediction")


In [37]:
auc = auc.evaluate(rfc_predictions)
auc

0.9144871380906253

In [39]:
gbt_predictions.printSchema()

root
 |-- features: vector (nullable = true)
 |-- PrivateIndex: double (nullable = false)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



In [40]:
# get the metrix

from pyspark.mllib.evaluation import MulticlassMetrics

In [69]:
predictionAndLabels = gbt_predictions.select(['prediction', 'PrivateIndex'])

predictionAndLabels.show()

+----------+------------+
|prediction|PrivateIndex|
+----------+------------+
|       0.0|         0.0|
|       0.0|         0.0|
|       0.0|         0.0|
|       0.0|         0.0|
|       0.0|         0.0|
|       0.0|         0.0|
|       0.0|         0.0|
|       0.0|         0.0|
|       0.0|         0.0|
|       0.0|         0.0|
|       0.0|         0.0|
|       0.0|         0.0|
|       0.0|         0.0|
|       0.0|         0.0|
|       0.0|         0.0|
|       0.0|         0.0|
|       0.0|         0.0|
|       0.0|         0.0|
|       0.0|         0.0|
|       0.0|         0.0|
+----------+------------+
only showing top 20 rows



In [70]:
metric = MulticlassMetrics(predictionAndLabels)

AttributeError: 'DataFrame' object has no attribute 'ctx'